In [1]:
from google.colab import drive 
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
#pip install imutils

  Using cached imutils-0.5.4.tar.gz (17 kB)
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25862 sha256=785904beebd60864f0a5912305c89a901bd599e5325e29bd394900125d36f95e
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\86\d7\0a\4923351ed1cec5d5e24c1eaf8905567b02a0343b24aa873df2
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd 
import re
import math
import os
import sys
import cv2
import glob
import time
import imutils
import numpy as np
#from google.colab.patches import cv2_imshow

In [2]:
path = 'D:/test/AIHub'

# Cut AIHub Only Violence Part

In [ ]:
mp4_files = glob.glob(path+'**/*.mp4', recursive=True)

In [4]:
pip install moviepy

  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110726 sha256=2c6b0517b2607afd3cce7c94083bfdb73aba990bcb368d46f8045d766676c4b9
  Stored in directory: c:\users\seogki\appdata\local\pip\cache\wheels\e4\a4\db\0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1

  Created wheel for proglog: filename=proglog-0.1.9-py3-none-any.whl size=6147 sha256=b2fc13a72dba062542f6be47ce6963c70e590607bd1774d1fa8c4d4a5e3a8621
  Stored in directory: c:\users\seogki\appdata\local\pip\cache\wheels\22\82\ff\56950e4b223a69eda3a2bdeaef2ae4b274c3d1ced9b5814c99
Successfully built moviepy proglog


In [23]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    name, ext = os.path.splitext(filename)
    if not targetname:
        T1, T2 = [int(1000*t) for t in [t1, t2]]
        targetname = "%sSUB%d_%d.%s" % (name, T1, T2, ext)

    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-ss", "%0.2f"%t1,
           "-i", filename,
           "-t", "%0.2f"%(t2-t1),
           "-vcodec", "copy", "-acodec", "copy", targetname]

    subprocess_call(cmd)

In [ ]:
def aihub_data_cut_violence(video_path, csv_path):
    mp4_files = glob.glob(video_path+'**/*.mp4', recursive=True)
    events = pd.read_csv(csv_path)
    out_path_list = list()

    for idx, i in enumerate(mp4_files):
        start = 0
        end = 0
        for j in events.itertuples():
            if (i.split('AIHub/')[1].split('_resized')[0] == j.fname.split('.')[0]):
                if (len(j.start.split(':')) == 3):
                    start = float(j.start.split(':')[0]) * 60 * 60 + float(j.start.split(':')[1]) * 60 + float(j.start.split(':')[2])
                    end = start + float(j.end.split(':')[0]) * 60 * 60 + float(j.end.split(':')[1]) * 60 + float(j.end.split(':')[2])
                else:
                    start = float(j.start.split(':')[0]) * 60 + float(j.start.split(':')[1]) 
                    end = start + float(j.end.split(':')[0]) * 60 + float(j.end.split(':')[1])
                break
        
        out = i.split('.')[0]+'_fi.mp4'
        # print(i,start,end,out)
        ffmpeg_extract_subclip(i, start, end, targetname=out)
        out_path_list.append(out)
        print('working... {}', round(idx/len(mp4_files),2))
    return out_path_list

#path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#csv_path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/events_time.csv'
#out_path = aihub_data_cut_violence(path, csv_path)

# Box Crop Video


In [ ]:
def box_crop_video_by_frame(video_path, out_path, box_width, box_height, object_position):
    '''
    video_path : input video path
    out_path : path for saving video
    box_width : width of box
    box_height : height of box
    object_position : list of object posisitons
    '''

    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 영상의 넓이(가로) 프레임
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 영상의 높이(세로) 프레임
    frameRate = int(cap.get(cv2.CAP_PROP_FPS)) 
    top,bottom,left,right = get_box_size(box_width, box_height, width, height, object_position) 
    
    # 비디오 저장 변수
    writer = None
    
    while True:
        # ret : 성공적으로 불러왔는지 확인
        # frame : 읽어온 frame 정보    
        ret, frame = cap.read()
        
        # 읽은 Frame이 없는 경우 종료
        if not ret:
            break

        # Frame Resize
        frame = frame[top:bottom,left:right,:]
        #cv2.imshow('frame', frame)
        #cv2_imshow(frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        
        # 저장할 video 설정
        if writer is None:
            fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # MJPG
            writer = cv2.VideoWriter(out_path, fourcc, frameRate, (frame.shape[1], frame.shape[0]), True)
        
        # 비디오 저장
        if writer is not None:
            writer.write(frame)
            
    print('Successfully cropped : ', out_path)
    return out_path

In [ ]:
def get_box_size(box_width, box_height, vid_width, vid_height, people):
    '''
    box_width : width of the box
    box_height : height of the box
    vid_width : width of input video
    vid_height :  height of input video
    people : list of object position [[x1,y1],[x2,y2], ... [xn,yn]]]

    returns cornor coordinate of box in the video
    '''
    print('cutting {}x{} in {}x{} video'.format(box_width, box_height, vid_width, vid_height))
    pos_wh = np.array([[x[0],x[1]] for x in people])
    center = pos_wh.mean(axis=0)
    print('center :', center)
    left = int(center[0] - box_width/2)
    top = int(center[1] -  box_height/2)
    if left > vid_width - box_width:
        left = vid_width - box_width 
    elif left < 0:
        left = 0

    if top > vid_height - box_height:
        top = vid_height - box_height 
    elif top < 0:
        top = 0

    right = left + box_width
    bottom = top + box_height

    print('box pixel area : {}, {}'.format((left,top),(right,bottom)))
    assert left<right and top<bottom, 'left, top has to be lower than right, bottom'
    assert right <= vid_width and bottom<=vid_height, 'box goes out of the video'
    return top,bottom,left,right

In [ ]:
aihub_pos = pd.read_csv('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/aihub_position.csv')
aihub_info = pd.read_csv('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/aihub_info.csv')

In [ ]:
aihub_info.shape, aihub_pos.shape

((1283, 8), (2830, 4))

In [ ]:
aihub_info.drop(labels=['width','height', 'depth','population'], axis=1, inplace=True)

In [ ]:
aihub_info.head()

,filename,eventname,start,duration
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,assault,90,200
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,assault,51,248
2,outsidedoor_01/11-2/11-2_cam01_assault01_place...,assault,92,200
3,outsidedoor_01/11-2/11-2_cam01_assault01_place...,assault,52,248
4,outsidedoor_01/12-3/12-3_cam02_assault01_place...,assault,81,163


In [ ]:
aihub_pos.head()

,filename,objname,x,y
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,2750,1325
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,3323,1164
2,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,2926,1473
3,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,3509,1146
4,outsidedoor_01/11-2/11-2_cam01_assault01_place...,person_1,721,1063


In [ ]:
# video was downscaled from 3840x2160 to 720x406
aihub_pos['x'] = (aihub_pos['x']/3840*720).map(lambda x: int(x))
aihub_pos['y'] = (aihub_pos['y']/2160*406).map(lambda x: int(x))

In [ ]:
aihub_pos.columns = ['filename','objname','width','height']

In [ ]:
aihub_pos.head()

,filename,objname,width,height
0,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,515,249
1,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,623,218
2,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_1,548,276
3,outsidedoor_01/11-2/11-2_cam02_assault01_place...,person_2,657,215
4,outsidedoor_01/11-2/11-2_cam01_assault01_place...,person_1,135,199


In [ ]:
def ffmpeg_extract_box_cropped_subclip(filename, box_width, box_height,left,top, targetname=None):
    cmd = [get_setting("FFMPEG_BINARY"),"-y",
           "-i", filename,
           "-vf", "crop={}:{}:{}:{}".format(box_width,box_height,left,top),
           targetname]

    subprocess_call(cmd)

In [ ]:
def box_crop(path, pos_df, box_width, box_height):
    '''
    works same as box_crop_ffmpeg but this function use cv2 to crop and this takes longer time
    SO DONT USE
    '''
    fight_vid_list = glob.glob(path + '**/*_resized_720_fi.mp4', recursive=True)
    cropped_video = list()

    for idx, in_path in enumerate(fight_vid_list):
        filename = in_path.split(path)[-1].split('_resized_720_fi.mp4')[0]

        obj_pos = list()
        for index, row in pos_df[pos_df['filename']==filename + '.xml'].iterrows():
            obj_pos.append([row.width, row.height])

        out_path = in_path.split('_fi.mp4')[0] + '_cropped_fi.mp4'
        print('working on : ',filename)
        cropped_video.append(box_crop_video_by_frame(in_path, out_path, box_width, box_height, obj_pos))
        print('---- working... {}% finished !!'.format(round(idx/len(fight_vid_list),2)))
        
    return cropped_video

path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#cropped_vid = box_crop(path, aihub_pos, 360, 360)

In [ ]:
# cropped_fi.mp4 파일 제거
key = str(input('Are you sure to delete all _fi.mp4 files?\ntype "yes" if so..\n> '))
if key == "yes":
    for vid in glob.glob(path + '**/*_cropped_fi.mp4', recursive=True):
        if os.path.isfile(vid):
            os.remove(vid)
        else:
            print(vid,' doesnt\'t exist')
    print('done')

Are you sure to delete all _fi.mp4 files?
type "yes" if so..
> no


In [ ]:
# _fi.mp4 파일 제거
key = str(input('Are you sure to delete all _fi.mp4 files?\ntype "yes" if so..\n> '))
if key == "yes":
    for vid in glob.glob(path + '**/*_cropped_fi.mp4', recursive=True):
        if os.path.isfile(vid):
            os.remove(vid)
        else:
            print(vid,' doesnt\'t exist')
    print('done')

Are you sure to delete all _fi.mp4 files?
type "yes" if so..
> no


In [ ]:
aihub_df = pd.merge(aihub_pos, aihub_info, left_on='filename', right_on='filename', how='outer')

In [ ]:
def box_crop_ffmpeg(path, pos_df, box_width, box_height):
    fight_vid_list = glob.glob(path + '**/*_resized_720_fi.mp4', recursive=True)
    cropped_video = list()

    for idx, in_path in enumerate(fight_vid_list):
        filename = in_path.split(path)[-1].split('_resized_720_fi.mp4')[0]
        out_path = in_path.split('_fi.mp4')[0] + '_cropped_fi.mp4'

        if os.path.isfile(out_path):
            continue

        obj_pos = list()
        start_time = time.time()
        start = 0
        end = 0
        
        for index, row in pos_df[pos_df['filename']==filename + '.xml'].iterrows():
            if not start or not end:
                start = row.start
                end = start + row.duration
            obj_pos.append([row.width, row.height])
        print(start, end)

        print('working on : ',filename)
        cap = cv2.VideoCapture(in_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # 영상의 넓이(가로) 프레임
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # 영상의 높이(세로) 프레임
        frameRate = int(cap.get(cv2.CAP_PROP_FPS)) 
        top,_,left,_ = get_box_size(box_width, box_height, width, height, obj_pos)         
        cap.release()

        ffmpeg_extract_box_cropped_subclip(in_path,box_width, box_height,left,top,out_path)
        cropped_video.append(out_path)
        print('---- working... {}%... {} out of {} finished !!\ntime took : {} sec'.format(round(idx+1/len(fight_vid_list) * 100,2), idx+1, len(fight_vid_list), round(time.time()-start_time,3)))

    return cropped_video

#path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#box_cropped_vid = box_crop_ffmpeg(path, aihub_df, 360, 360)

# non-fight data subclip extract

In [ ]:
# cropped_fi.mp4 파일 제거
key = str(input('Are you sure to delete all _fi.mp4 files?\ntype "yes" if so..\n> '))
if key == "yes":
    for vid in glob.glob(path + '**/*.xml', recursive=True):
        if os.path.isfile(vid):
            os.remove(vid)
        else:
            print(vid,' doesnt\'t exist')
    print('done')

Are you sure to delete all _fi.mp4 files?
type "yes" if so..
> yes
done


In [4]:
# still working on...
import json

#path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/'
#save_fi_path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub_Dataset/fight/'
#save_nofi_path = '/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub_Dataset/noFight/'
save_nofi_path = 'E:/test/cropped_nofight'
#save_nofi_path = 'D:/test/AIHub_Dataset/noFight/'

In [5]:
json_path = 'E:/test/action_info_filepath.json'
with open(json_path, "r") as st_json:
    aihub_action_data = json.load(st_json)

FileNotFoundError: [Errno 2] No such file or directory: 'E:/test/action_info_filepath.json'

In [11]:
aihub_action_data

[{'action_dict': {'kicking': [[3372, 3434],
    [4314, 4400],
    [4614, 4722],
    [4929, 5049],
    [6391, 6465]],
   'punching': [[6746, 6955], [7378, 7639], [5410, 5469], [5499, 5575]],
   'pushing': [[2149, 2245], [2326, 2433], [2492, 2551], [2860, 2933]],
   'threaten': [[5696, 5749], [5907, 5951], [7041, 7098], [7182, 7234]],
   'throwing': [[3652, 3719], [5956, 6000]]},
  'fname': '/content/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub/inside_croki_02/24-5/24-5_cam03_assault01_place09_night_summer.xml'},
 {'action_dict': {'kicking': [[3545, 3604],
    [4463, 4520],
    [4774, 4876],
    [5101, 5196],
    [6378, 6447]],
   'punching': [[5482, 5545],
    [5570, 5633],
    [6657, 6845],
    [7228, 7522],
    [7747, 7810]],
   'pushing': [[2198, 2273], [2397, 2498], [2537, 2599], [2948, 3021]],
   'threaten': [[5700, 5750],
    [5852, 5900],
    [6903, 6963],
    [7056, 7128],
    [7846, 7912]],
   'throwing': [[3696, 3762], [6011, 6046]]},
  'fname': '/content/gd

In [67]:
# fight clip extraction according to action data
fps = 30

video_length = len(aihub_action_data)
cnt = 1

video_not_found = list()

for video in aihub_action_data:
    filepath = video['fname'].split('/content')[1].split('.xml')[0] + '_resized_720.mp4'
    filepath = path+'/'+filepath.split('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub')[-1]
    if not os.path.isfile(filepath):
        video_not_found.append(filepath)
        cnt += 1
        continue
    filename = os.path.basename(filepath).split('_resized_720.mp4')[0]
    path_dir = filepath.split('/')
    action_dict = video['action_dict']
    index = 1

    start_time = time.time()
    for action, action_frame in action_dict.items():
        if action == 'threaten' or action == 'falldown':
            continue
        else:
            for start, end in action_frame:
                start = start / fps
                end = end / fps
                if end-start < 2.0:
                    start, end = start-0.5, end+0.5
                save_name = save_fi_path +'AA_V_AH_'+ path_dir[-3] + '_' + path_dir[-2] + '_' + filename + '_' + str(index) +'.mp4'
                index += 1
                if os.path.isfile(save_name):
                    continue
                ffmpeg_extract_subclip(filepath,start,end,save_name)

    print('time took : ',round(time.time() - start_time,2), ' sec')
    print('{} finished!'.format(filepath))
    print('{}/{} done / {}% finished'.format(cnt,video_length,round((cnt/video_length)*100,2)))
    cnt+=1

NameError: name 'aihub_action_data' is not defined

In [68]:
# fight clip extraction according to action data
fps = 30

video_length = len(aihub_action_data)
cnt = 1

video_not_found = list()

for video in aihub_action_data:
    filepath = video['fname'].split('/content')[1].split('.xml')[0] + '_resized_720.mp4'
    filepath = path+'/'+filepath.split('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub')[-1]
    if not os.path.isfile(filepath):
        video_not_found.append(filepath)
        cnt += 1
        continue
    filename = os.path.basename(filepath).split('_resized_720.mp4')[0]
    path_dir = filepath.split('/')
    action_dict = video['action_dict']
    index = 1

    start_time = time.time()
    for action, action_frame in action_dict.items():
        if action == 'threaten' or action == 'falldown':
            continue
        else:
            for start, end in action_frame:
                start = start / fps
                end = end / fps
                if end-start < 2.0:
                    start, end = start-0.5, end+0.5
                save_name = save_fi_path +'AA_V_AH_'+ path_dir[-3] + '_' + path_dir[-2] + '_' + filename + '_' + str(index) +'.mp4'
                index += 1
                if os.path.isfile(save_name):
                    continue
                ffmpeg_extract_subclip(filepath,start,end,save_name)

    print('time took : ',round(time.time() - start_time,2), ' sec')
    print('{} finished!'.format(filepath))
    print('{}/{} done / {}% finished'.format(cnt,video_length,round((cnt/video_length)*100,2)))
    cnt+=1

NameError: name 'aihub_action_data' is not defined

In [11]:
import pandas as pd

In [16]:
import json
save_nofi_path = 'D:/test/nofight/'


In [7]:
aihub_action_start_data = pd.read_csv('D:/test/aihub_fight_start.csv')

In [8]:
aihub_action_start_data.drop(columns='Unnamed: 0', inplace=True)

In [9]:
aihub_action_start_data.columns = ['fname','start_frame']

In [10]:
aihub_action_start_data.head()

,fname,start_frame
0,/content/gdrive/Shareddrives/2021청년인재_ᄀ...,2149
1,/content/gdrive/Shareddrives/2021청년인재_ᄀ...,2198
2,/content/gdrive/Shareddrives/2021청년인재_ᄀ...,2290
3,/content/gdrive/Shareddrives/2021청년인재_ᄀ...,2288
4,/content/gdrive/Shareddrives/2021청년인재_ᄀ...,2143


In [74]:
# non fight clip extraction according to action data
fps = 30

video_count = len(aihub_action_start_data)
cnt = 1
outside_cnt = 0
video_not_found2 = list()
vid_not_enough = list()

for video in aihub_action_start_data.iterrows():
    video = video[1]
    start_time = time.time()
    filepath = video['fname'].split('/content')[1].split('.xml')[0] + '_resized_720.mp4'
    filepath = path+'/'+filepath.split('/gdrive/Shareddrives/2021청년인재_고려대과정_10조/Data/AIHub')[-1]
    if not os.path.isfile(filepath):
        video_not_found2.append(filepath)
        cnt += 1
        continue
    if video['start_frame'] < 2000:
        vid_not_enough.append(filepath)
        cnt += 1
        continue
    #print(video['start_frame'])
    mid = video['start_frame']
    start = 0
    end = 2000 /fps

    filename = os.path.basename(filepath).split('_resized_720.mp4')[0]
    path_dir = filepath.split('/')
    save_name = save_nofi_path +'AA_N_AH_'+ path_dir[-3] + '_' + path_dir[-2] + '_' + filename + '.mp4'
    if os.path.isfile(save_name):
        continue
    #print(filepath,start,end,save_name)
    
    if 'outsidedoor' in filepath:
        outside_cnt += 1
        ffmpeg_extract_subclip(filepath,start,end,save_name)

    print('time took : ',round(time.time() - start_time,2), ' sec')
    print('{} finished!'.format(filepath))
    print('{}/{} done / {}% finished'.format(cnt,video_count,round((cnt/video_count)*100,2)))
    cnt+=1

time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam03_assault01_place09_night_summer_resized_720.mp4 finished!
1/1229 done / 0.08% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam03_assault01_place09_night_spring_resized_720.mp4 finished!
2/1229 done / 0.16% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam02_assault01_place09_night_winter_resized_720.mp4 finished!
3/1229 done / 0.24% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam01_assault01_place09_night_winter_resized_720.mp4 finished!
4/1229 done / 0.33% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam01_assault01_place09_night_summer_resized_720.mp4 finished!
5/1229 done / 0.41% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam03_assault01_place09_night_winter_resized_720.mp4 finished!
6/1229 done / 0.49% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_02/24-5/24-5_cam01_a

time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam02_assault01_place09_night_winter_resized_720.mp4 finished!
265/1229 done / 21.56% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam03_assault01_place09_night_summer_resized_720.mp4 finished!
266/1229 done / 21.64% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam02_assault01_place09_night_summer_resized_720.mp4 finished!
267/1229 done / 21.72% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam01_assault01_place09_night_winter_resized_720.mp4 finished!
268/1229 done / 21.81% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam03_assault01_place09_night_winter_resized_720.mp4 finished!
269/1229 done / 21.89% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_04/412-3/412-3_cam02_assault01_place09_night_spring_resized_720.mp4 finished!
270/1229 done / 21.97% finished
time took :  0.0  sec
D:/test/AIHub//ins

time took :  0.0  sec
D:/test/AIHub//inside_croki_03/407-5/407-5_cam03_assault01_place04_day_summer_resized_720.mp4 finished!
512/1229 done / 41.66% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_03/407-5/407-5_cam01_assault01_place04_day_summer_resized_720.mp4 finished!
513/1229 done / 41.74% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_03/407-5/407-5_cam02_assault01_place04_day_winter_resized_720.mp4 finished!
515/1229 done / 41.9% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_03/407-5/407-5_cam02_assault01_place04_day_summer_resized_720.mp4 finished!
516/1229 done / 41.99% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_03/407-5/407-5_cam03_assault01_place04_day_spring_resized_720.mp4 finished!
517/1229 done / 42.07% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_03/411-3/411-3_cam03_assault01_place08_night_summer_resized_720.mp4 finished!
520/1229 done / 42.31% finished
time took :  0.0  sec
D:/test/AIHub//inside_croki_0

time took :  0.0  sec
D:/test/AIHub//insidedoor_10/45-3/45-3_cam02_assault01_place01_day_spring_resized_720.mp4 finished!
727/1229 done / 59.15% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam01_assault01_place03_day_winter_resized_720.mp4 finished!
728/1229 done / 59.24% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam01_assault01_place03_day_spring_resized_720.mp4 finished!
731/1229 done / 59.48% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam02_assault01_place03_day_winter_resized_720.mp4 finished!
732/1229 done / 59.56% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam02_assault01_place03_day_spring_resized_720.mp4 finished!
733/1229 done / 59.64% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam03_assault01_place03_day_spring_resized_720.mp4 finished!
734/1229 done / 59.72% finished
time took :  0.0  sec
D:/test/AIHub//insidedoor_10/439-5/439-5_cam

Moviepy - Command successful
time took :  1.4  sec
D:/test/AIHub//outsidedoor_01/11-6/11-6_cam01_assault01_place08_night_summer_resized_720.mp4 finished!
788/1229 done / 64.12% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.44  sec
D:/test/AIHub//outsidedoor_01/12-2/12-2_cam01_assault01_place09_day_summer_resized_720.mp4 finished!
789/1229 done / 64.2% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.38  sec
D:/test/AIHub//outsidedoor_01/12-2/12-2_cam02_assault01_place09_day_summer_resized_720.mp4 finished!
790/1229 done / 64.28% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.39  sec
D:/test/AIHub//outsidedoor_01/12-2/12-2_cam02_assault01_place09_day_spring_resized_720.mp4 finished!
791/1229 done / 64.36% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.41  sec
D:/test/AIHub//outsidedoor_01/12-2/12-2_cam01_as

...

Moviepy - Command successful
time took :  1.37  sec
D:/test/AIHub//outsidedoor_03/17-5/17-5_cam02_assault03_place03_night_summer_resized_720.mp4 finished!
880/1229 done / 71.6% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.37  sec
D:/test/AIHub//outsidedoor_03/17-6/17-6_cam01_assault03_place03_night_spring_resized_720.mp4 finished!
881/1229 done / 71.68% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.35  sec
D:/test/AIHub//outsidedoor_03/17-6/17-6_cam02_assault03_place03_night_summer_resized_720.mp4 finished!
883/1229 done / 71.85% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.38  sec
D:/test/AIHub//outsidedoor_03/17-6/17-6_cam02_assault03_place03_night_spring_resized_720.mp4 finished!
884/1229 done / 71.93% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.34  sec
D:/test/AIHub//outsidedoor_04/18-1/18-1_c

Moviepy - Command successful
time took :  1.37  sec
D:/test/AIHub//outsidedoor_06/23-1/23-1_cam01_assault01_place02_night_summer_resized_720.mp4 finished!
947/1229 done / 77.05% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.5  sec
D:/test/AIHub//outsidedoor_06/23-1/23-1_cam01_assault01_place02_night_spring_resized_720.mp4 finished!
948/1229 done / 77.14% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.45  sec
D:/test/AIHub//outsidedoor_06/23-1/23-1_cam02_assault01_place02_night_summer_resized_720.mp4 finished!
949/1229 done / 77.22% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.36  sec
D:/test/AIHub//outsidedoor_06/23-2/23-2_cam02_assault01_place02_night_spring_resized_720.mp4 finished!
950/1229 done / 77.3% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.42  sec
D:/test/AIHub//outsidedoor_06/23-2/23-2_ca

Moviepy - Command successful
time took :  1.36  sec
D:/test/AIHub//outsidedoor_07/4-2/4-2_cam01_assault01_place01_day_spring_resized_720.mp4 finished!
995/1229 done / 80.96% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.36  sec
D:/test/AIHub//outsidedoor_07/4-2/4-2_cam02_assault01_place01_day_summer_resized_720.mp4 finished!
996/1229 done / 81.04% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.37  sec
D:/test/AIHub//outsidedoor_07/4-2/4-2_cam02_assault01_place01_day_spring_resized_720.mp4 finished!
997/1229 done / 81.12% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.33  sec
D:/test/AIHub//outsidedoor_07/4-4/4-4_cam02_assault01_place01_day_spring_resized_720.mp4 finished!
998/1229 done / 81.2% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.35  sec
D:/test/AIHub//outsidedoor_07/4-4/4-4_cam01_assault01_pla

Moviepy - Command successful
time took :  1.39  sec
D:/test/AIHub//outsidedoor_08/406-5/406-5_cam01_assault01_place03_day_spring_resized_720.mp4 finished!
1032/1229 done / 83.97% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.42  sec
D:/test/AIHub//outsidedoor_08/406-5/406-5_cam01_assault01_place03_day_summer_resized_720.mp4 finished!
1033/1229 done / 84.05% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.46  sec
D:/test/AIHub//outsidedoor_08/406-6/406-6_cam01_assault01_place03_day_summer_resized_720.mp4 finished!
1034/1229 done / 84.13% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.56  sec
D:/test/AIHub//outsidedoor_08/406-6/406-6_cam01_assault01_place03_day_spring_resized_720.mp4 finished!
1035/1229 done / 84.21% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.62  sec
D:/test/AIHub//outsidedoor_08/406-6/

Moviepy - Command successful
time took :  1.37  sec
D:/test/AIHub//outsidedoor_08/5-6/5-6_cam02_assault01_place08_day_summer_resized_720.mp4 finished!
1069/1229 done / 86.98% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.41  sec
D:/test/AIHub//outsidedoor_08/7-2/7-2_cam01_assault01_place04_day_summer_resized_720.mp4 finished!
1074/1229 done / 87.39% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.39  sec
D:/test/AIHub//outsidedoor_08/7-2/7-2_cam02_assault01_place04_day_summer_resized_720.mp4 finished!
1075/1229 done / 87.47% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.38  sec
D:/test/AIHub//outsidedoor_09/7-4/7-4_cam01_assault01_place04_day_spring_resized_720.mp4 finished!
1082/1229 done / 88.04% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
time took :  1.39  sec
D:/test/AIHub//outsidedoor_09/7-4/7-4_cam02_assault0

In [19]:
len(vid_not_enough)

195

In [21]:
len(video_not_found2)

73

In [75]:
outside_cnt

261

In [25]:
save_cropped_nofi_path = 'D:/test/cropped_nofight/'

In [27]:
import numpy as np

In [57]:
def get_rand_time():
    return np.random.rand() * 2 + 3

In [58]:
get_rand_time()

4.034650406220494

In [76]:
cnt = 0 
temp = glob.glob(save_nofi_path+'**/*.mp4',recursive=True)
for video in temp:
    #print(video)
    index = 1
    name = os.path.basename(video).split('.mp4')[0]
    prev = 0
    next = prev + get_rand_time()
    while next<66.6:
        next = prev + get_rand_time()
        out_path = 'D:/test/cropped_nofight/' + name + '_' + str(index) + '.mp4'
        #print(video, prev, next , out_path)
        ffmpeg_extract_subclip(video, prev, next , out_path)
        index += 1
        prev = next
    cnt+=1
    print('{}/{} done / {}% finished'.format(cnt,len(temp),round((cnt/len(temp))*100,2)))

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
7/261 done / 2.68% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviep

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
14/261 done / 5.36% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
21/261 done / 8.05% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Movie

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
28/261 done / 10.73% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Movi

...

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
231/261 done / 88.51% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Mov

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
238/261 done / 91.19% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Mov

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Movie

Moviepy - Command successful
251/261 done / 96.17% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Mov

Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
258/261 done / 98.85% finished
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Moviepy - Running:
>>> "+ " ".join(cmd)
Mov